# Feature Engineering
- In this notebook, we calculate some features that have to do with the "talent level" of a team. This is captured to some unknown (due to proprietary algorithm) degree in the FPI, but can be more explicitely calculated using the recruiting ratings as we will see below.
- We will also calculate blue chip ratios using recruiting rankings. This is the fraction of a roster that is 4 and 5 star recruits. For many teams, this value is 0, but for the top 50 or so teams, this is correlated substantively with win percentage.


In [2]:
#!pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 8.5 MB/s eta 0:00:00:00:0100:01


In [4]:
# Get general dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [38]:
player_info_df = pd.read_csv('data/team_info.csv')
team_recruiting_df = pd.read_csv('data/team_recruiting.csv')
player_recruiting_df = pd.read_csv('data/player_recruiting.csv').rename(columns={'school': 'team'})

We'll stick with data from 2007 on since that's where 247Sports Composite ranking reliably ranked most recruits to major teams.

In [39]:
# Filter out non FBS teams
fbs_teams = player_info_df['team'].tolist()
team_recruiting_df = team_recruiting_df[team_recruiting_df['team'].isin(fbs_teams)]

# Keep years from 2007 on when rankings reliable
team_recruiting_df = team_recruiting_df[team_recruiting_df['year'] >= 2007]
team_recruiting_df.head()

,team,year,rank,points
782,Florida,2007,1,313.05
783,USC,2007,2,295.06
784,Texas,2007,3,293.10
785,Tennessee,2007,4,284.42
786,LSU,2007,5,282.25


In [40]:
player_recruiting_df = player_recruiting_df[player_recruiting_df['year'] >= 2007]
player_recruiting_df.head()

,name,year,star,team,state,ranking,rating
10573,Joe McKnight,2007,5,USC,LA,1.0,0.9997
10574,Jimmy Clausen,2007,5,Notre Dame,CA,2.0,0.9987
10575,Eric Berry,2007,5,Tennessee,GA,3.0,0.9985
10576,Marvin Austin,2007,5,North Carolina,DC,4.0,0.9977
10577,Ryan Mallett,2007,5,Michigan,TX,5.0,0.9976


### Rolling average of 4 recruiting classes for each team

In [46]:
# Calculate rolling average of team recruiting class rating
team_recruiting_df['talent_level'] = team_recruiting_df.groupby(by=['team'])['points'].rolling(window=4, min_periods=4).mean().reset_index(level=0, drop=True)

# There will be some NaN values. Count them
print(f"There are {team_recruiting_df[team_recruiting_df.year == 2010]['talent_level'].isna().sum()} null values in 2010")

# Fill these null values with talent level of 0
team_recruiting_df['talent_level'] = team_recruiting_df['talent_level'].fillna(0)

# Keep only records from 2010 where we'll have non zero talent level
#team_recruiting_2010_df = team_recruiting_df[team_recruiting_df['year'] >= 2010].copy()

team_recruiting_df

There are 7 null values in 2010


,team,year,rank,points,talent_level
782,Florida,2007,1,313.05,0.0000
783,USC,2007,2,295.06,0.0000
784,Texas,2007,3,293.10,0.0000
785,Tennessee,2007,4,284.42,0.0000
786,LSU,2007,5,282.25,0.0000
...,...,...,...,...,...
4064,New Mexico State,2024,134,67.75,107.0550
4080,Army,2024,152,37.76,74.3475
4081,Navy,2024,153,37.64,69.4150
4099,Air Force,2024,172,23.21,102.0175


### Blue Chip Ratios for Each Team
- Start using data in 2007 to calculate BCR in 2010 and beyond

In [47]:
blue_chip_counts = player_recruiting_df[player_recruiting_df.star >= 4].groupby(['year','team'])['star'].size()
blue_chip_counts.head()

year  team         
2007  Alabama          6
      Arizona          1
      Arizona State    1
      Arkansas         1
      Auburn           8
Name: star, dtype: int64

In [48]:
total_player_counts = player_recruiting_df.groupby(['year','team'])['star'].size()
total_player_counts.head()

year  team         
2007  Air Force         8
      Akron            20
      Alabama          21
      Arizona          10
      Arizona State    18
Name: star, dtype: int64

In [49]:
blue_chip_df = team_recruiting_df.merge(blue_chip_counts, 
                    how='left', on=['year', 'team']).merge(total_player_counts, 
                                                           how='left', on=['year', 'team'])
blue_chip_df = blue_chip_df.rename(columns={'star_x': 'blue_chip', 'star_y': 'total'})
blue_chip_df[['blue_chip', 'total']] = blue_chip_df[['blue_chip', 'total']].fillna(0)
blue_chip_df

,team,year,rank,points,talent_level,blue_chip,total
0,Florida,2007,1,313.05,0.0000,18.0,23.0
1,USC,2007,2,295.06,0.0000,14.0,18.0
2,Texas,2007,3,293.10,0.0000,16.0,24.0
3,Tennessee,2007,4,284.42,0.0000,10.0,27.0
4,LSU,2007,5,282.25,0.0000,14.0,24.0
...,...,...,...,...,...,...,...
2313,New Mexico State,2024,134,67.75,107.0550,0.0,0.0
2314,Army,2024,152,37.76,74.3475,0.0,0.0
2315,Navy,2024,153,37.64,69.4150,0.0,0.0
2316,Air Force,2024,172,23.21,102.0175,0.0,0.0


In [50]:
# Calculate running sums for blue chip ratio
blue_chip_df['blue_sums'] = blue_chip_df.groupby('team')['blue_chip'].rolling(window=4, min_periods=4).sum().reset_index(level=0, drop=True)
blue_chip_df['total_sums'] = blue_chip_df.groupby('team')['total'].rolling(window=4, min_periods=4).sum().reset_index(level=0, drop=True)
blue_chip_df['blue_chip_ratio'] = blue_chip_df['blue_sums']/blue_chip_df['total_sums']

# Filter out until after 2010 to get relevant results
blue_chip_2010_df = blue_chip_df[(blue_chip_df.year >= 2010) ].copy()
blue_chip_2010_df['blue_chip_ratio'] = blue_chip_2010_df['blue_chip_ratio'].fillna(0)

blue_chip_2010_df

,team,year,rank,points,talent_level,blue_chip,total,blue_sums,total_sums,blue_chip_ratio
346,Florida,2010,1,324.62,298.5900,22.0,28.0,64.0,83.0,0.771084
347,Texas,2010,2,312.07,288.4725,21.0,24.0,67.0,88.0,0.761364
348,USC,2010,3,294.73,292.3525,15.0,17.0,57.0,70.0,0.814286
349,Alabama,2010,4,284.20,276.5550,14.0,22.0,61.0,100.0,0.610000
350,Oklahoma,2010,5,283.58,262.9475,17.0,30.0,46.0,88.0,0.522727
...,...,...,...,...,...,...,...,...,...,...
2313,New Mexico State,2024,134,67.75,107.0550,0.0,0.0,0.0,31.0,0.000000
2314,Army,2024,152,37.76,74.3475,0.0,0.0,0.0,61.0,0.000000
2315,Navy,2024,153,37.64,69.4150,0.0,0.0,0.0,39.0,0.000000
2316,Air Force,2024,172,23.21,102.0175,0.0,0.0,0.0,75.0,0.000000


In [51]:
# Check results for known team
blue_chip_2010_df[blue_chip_2010_df.team == 'Ohio State']

,team,year,rank,points,talent_level,blue_chip,total,blue_sums,total_sums,blue_chip_ratio
363,Ohio State,2010,18,237.38,229.8300,10.0,16.0,44.0,61.0,0.721311
472,Ohio State,2011,6,278.47,267.2025,13.0,25.0,52.0,80.0,0.650000
597,Ohio State,2012,5,286.13,270.9775,16.0,23.0,55.0,89.0,0.617978
725,Ohio State,2013,2,303.35,276.3325,19.0,24.0,58.0,88.0,0.659091
858,Ohio State,2014,3,296.08,291.0075,16.0,23.0,64.0,95.0,0.673684
995,Ohio State,2015,7,279.60,291.2900,15.0,26.0,66.0,96.0,0.687500
1125,Ohio State,2016,4,289.12,292.0375,17.0,25.0,67.0,98.0,0.683673
1256,Ohio State,2017,2,312.14,294.2350,19.0,22.0,67.0,96.0,0.697917
1389,Ohio State,2018,2,317.06,299.4800,22.0,26.0,73.0,99.0,0.737374
1534,Ohio State,2019,14,261.18,294.8750,12.0,17.0,70.0,90.0,0.777778


In [52]:
blue_chip_2010_df.to_csv('data/team_recruiting_w_blue_chip_ratios.csv', index=False)

### Coaching Prior Win Pct

In [33]:
coaches_df = pd.read_csv('data/coaches.csv')
coaches_df.head()

,name,team,year,games,wins,losses
0,Eli Abbott,Alabama,1902,8,4,4
1,Earl Abell,Colgate,1928,9,6,3
2,Earl Abell,Virginia,1929,9,4,3
3,Earl Abell,Virginia,1930,10,4,6
4,Earl Able,Mississippi State,1923,9,5,2


In [34]:
coaches_df.year.max()

2023

In [35]:
# Don't include 2023
#coaches_df = coaches_df[coaches_df.year <= 2023].sort_values(by='year').copy()
coaches_df = coaches_df.sort_values(by='year').copy()
# Group by name and create rolling average up to a given year.
coaches_df['cumulative_wins'] = coaches_df.groupby('name')['wins'].cumsum()
coaches_df['cumulative_games'] = coaches_df.groupby('name')['games'].cumsum()
coaches_df['career_win_pct'] = coaches_df['cumulative_wins']/coaches_df['cumulative_games']

# Fill Null values with 0 for new head coaches in a given year
coaches_df['career_win_pct'] = coaches_df['career_win_pct'].fillna(0)

In [36]:
coaches_df.to_csv('data/coach_career_win_pct.csv', index=False)